In [19]:
import bs4 as bs
import pandas as pd
import matplotlib as plt
from matplotlib import pyplot as pplt
import os 
import pprint as p

#Die Buch-Kürzel des alten Testaments
AT = ["Gn", "Ex", "Lv", "Nm", "Dt", "Ios", "Idc", "Rt", "1Sm", "2Sm", "3Rg", "4Rg", "1Par", "2Par", "Esr", "Neh", "Tb",
      "Idt", "Est", "1Mcc"
    , "2Mcc", "3Mcc", "4Mcc", "Iob", "Ps", "Prv", "Ecl", "Ct", "Sap", "Sir", "Is", "Ier", "Lam", "Bar", "Ez", "Dn",
      "Os", "Ioel", "Am", "Abd"
    , "Ion", "Mi", "Na", "Hab", "So", "Agg", "Za", "Mal"]


  
def percent_calc(num1, num2):
    '''returns percentage value'''
    return (num1/num2)*100
    
 
    
def analyse(filename):
    '''Opens a file specified by the filename, including the directory and creates dataframe for the respective file. Subsequently, this function analyses the chapter with regard to our criteria'''
    file_title = filename.split(".")[0]
    with open(filename, 'r') as f:
        file = f.read()   
    soup = bs.BeautifulSoup(file, 'lxml')
    Refs = soup.select('ref[cRef]')

    data = []
    for ref in Refs:
        if ref.get('cref').split('_')[0] in AT:
            if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
                data.append((ref.get('cref') , 'at', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
            else:
                data.append((ref.get('cref'), 'at', ref.getText(), 'not a direct quote'))
        else:
            if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
                data.append((ref.get('cref') , 'nt', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
            else:
                data.append((ref.get('cref'), 'nt', ref.getText(), 'not a direct quote'))
    #print(data)
    # der eigentliche Dataframe wird gesetzt
    df_data = pd.DataFrame(data)
    # der Header wird gesetzt.
    df_data.columns = ['kuerzel', 'atnt', 'stelle', 'zitat']



    # Hier wird eine Liste der wörtlichen Zitate erstellt
    qs = []
    for ref in Refs:
        if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling("q"):
            qs.append(ref.parent.find_previous_sibling("q").get_text(" ", strip=True))


    # Zählen der Gesamtlänge des zitierten Textes; one Leerzeilen und Zeilenumbrüche 
    quote_len = 0
    for i in range(len(qs)):
        quote_len +=len(qs[i].replace(" ","").replace("\n", ""))
        i+=1



    ges = bs.BeautifulSoup(file, 'lxml')

    ges_soup = ges.select('div')
    total_len=0
    for i in range(len(ges_soup)):
        total_len += len(ges_soup[i].getText().replace(" ", "").replace("n", ""))
    
    print(f"Anzahl an erfassten Bibelstellen: {len(Refs)}")
    print(f"wörtliche Zitate: {len(qs)}")
    print("Gesamtlänge:" + str(total_len))
    print("Zitatlänge :"+str(quote_len))    
    print("Prozentualer Anteil: " +(str(percent_calc(quote_len, total_len))))
    print(df_data["kuerzel"].value_counts().plot(figsize=(8,8), kind="pie", legend=False, title=f"{filename}"))
    pplt.savefig(f'Visualisierungen/{file_title}.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    print(df_data["atnt"].value_counts().plot(figsize=(8,8), kind="bar", legend=False, title=f"{filename}"))
    pplt.savefig(f'Visualisierungen/{file_title}_atnt.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    print(df_data["kuerzel"].value_counts()[:5].plot(figsize=(8,8), kind="barh", legend=False, title=f"{filename}  @cRef)", color="purple"))
    pplt.savefig(f'Visualisierungen/{file_title}_cref.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    print(df_data["stelle"].value_counts()[:10].plot(figsize=(8,8), kind="barh", legend=False, title=f"{filename} nach Stellen", color="red"))
    pplt.savefig(f'Visualisierungen/{file_title}_nach_stellen.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    

In [12]:
#Probe-Sachen
with open(f'interim_26/ch_02.xml', 'r') as f:
    file = f.read()
soup = bs.BeautifulSoup(file, 'lxml')
Refs = soup.select('ref[cRef]')

# Eine leere Liste wird erstellt und mit den Bibelstellen gefüllt. Hierbei wird sortiert, ob sie im Alten Testament stehen oder nicht.
# Ein zweites Auswahlkriterium ist die Frage, ob direkt vor dem Eltern-ELement des <ref type='biblical'> ein <q> steht, und somit ein direktes Zitat vorliegt.
data = []
for ref in Refs:
    if ref.get('cref').split('_')[0] in AT:
        if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
            data.append((ref.get('cref') , 'at', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
        else:
            data.append((ref.get('cref'), 'at', ref.getText(), 'not a direct quote'))
    else:
        if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
            data.append((ref.get('cref') , 'nt', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
        else:
            data.append((ref.get('cref'), 'nt', ref.getText(), 'not a direct quote'))
print(data)
# der eigentliche Dataframe wird gesetzt
df_data = pd.DataFrame(data)
# der Header wird gesetzt.
df_data.columns = ['kuerzel', 'atnt', 'stelle', 'zitat']



stellen = []
for i in range (0, len(df_data)):
    #if df_data['zitat'][i] != 'not a direct quote':
    stellen.append(df_data['stelle'][i])


# Hier wird eine Liste der wörtlichen Zitate erstellt
qs = []
for ref in Refs:
    if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling("q"):
        qs.append(ref.parent.find_previous_sibling("q").get_text(" ", strip=True))


# Zählen der Gesamtlänge des zitierten Textes; one Leerzeilen und Zeilenumbrüche
quote_len = 0
for i in range(len(qs)):
    quote_len +=len(qs[i].replace(" ","").replace("\n", ""))
    i+=1



ges = bs.BeautifulSoup(file, 'lxml')

ges_soup = ges.select('div')
total_len=0
for i in range(len(ges_soup)):
    total_len += len(ges_soup[i].getText().replace(" ", "").replace("n", ""))
print(f"Anzahl an erfassten Bibelstellen: {len(Refs)}")
print(f"wörtliche Zitate: {len(qs)}")
print("Gesamtlänge:" + str(total_len))
print("Zitatlänge :"+str(quote_len))
print("Prozentualer Anteil: " +(str(percent_calc(quote_len, total_len))))



[('Io_3,16-17', 'nt', 'Joh 3,16f.', 'not a direct quote'), ('Rm_3,9-20', 'nt', 'Röm 3,9-20', 'not a direct quote'), ('Eph_1,7', 'nt', 'Eph 1,7', 'not a direct quote'), ('Is_53,5-6', 'at', 'Jes 53,5f.', 'not a direct quote'), ('1Pt_2,24', 'nt', '1 Petr 2,24', 'not a direct quote'), ('2Cor_5,19', 'nt', '2 Kor 5,19', 'was warhafftig inn Christo und versönet die welt mit im selber und rechnete inen ire sünden Mehlhausen , Augsburger Interim , S.\xa042 : sünde. nicht zu unnd hat under uns auffgericht das wort von der versunung.'), ('Rm_3,25-26', 'nt', 'Röm 3,25f.', 'not a direct quote'), ('1Cor_1,29-31', 'nt', '1 Kor 1,29-31', 'not a direct quote')]
Anzahl an erfassten Bibelstellen: 8
wörtliche Zitate: 1
Gesamtlänge:1867
Zitatlänge :173
Prozentualer Anteil: 9.266202463845742


In [10]:

print(df_data["kuerzel"].value_counts().plot(figsize=(8,8), kind="pie", legend=False, title=f"02"))


NameError: name 'df_data' is not defined

In [39]:
analyse('interim_26/ch_25.xml')

ValueError: Length mismatch: Expected axis has 0 elements, new values have 4 elements